In [1]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

# make_imc_data was changed and sparsify_with_mask was added
from sgimc.utils import make_imc_data, sparsify, sparsify_with_mask

from sgimc import IMCProblem

from sgimc.qa_objective import QAObjectiveL2Loss
from sgimc.qa_objective import QAObjectiveLogLoss
from sgimc.qa_objective import QAObjectiveHuberLoss

from sgimc.algorithm.admm import sub_0_cg
from sgimc.algorithm.admm import sub_0_lbfgs
from sgimc.algorithm.admm import sub_m

from sgimc import imc_descent
from sgimc.utils import performance

from sgimc.utils import plot_WH, plot_loss

# loss calculation and boolean mask invertion
from utils import calculate_loss, invert

PROBLEM = "classification" if False else "regression"
random_state = np.random.RandomState(0x0BADCAFE)

In [2]:
from sgimc.algorithm import admm_step
from sgimc.algorithm.decoupled import step as decoupled_step

def step_qaadmm(problem, W, H, C, eta, method="l-bfgs", sparse=True,
                n_iterations=50, rtol=1e-5, atol=1e-8):

    approx_type = "quadratic" if method in ("cg",) else "linear"
    Obj = problem.objective(W, H, approx_type=approx_type)

    return admm_step(Obj, W, C, eta, sparse=sparse, method=method,
                     n_iterations=n_iterations, rtol=rtol, atol=atol)

def step_decoupled(problem, W, H, C, eta, rtol=1e-5, atol=1e-8):

    Obj = problem.objective(W, H, approx_type="linear")

    return decoupled_step(Obj, W, C, eta, rtol=rtol, atol=atol)

In [3]:
step_fn = step_qaadmm

if PROBLEM == "classification":
    QAObjectiveLoss = QAObjectiveLogLoss
else:
    QAObjectiveLoss = QAObjectiveL2Loss  # QAObjectiveHuberLoss

In [4]:
if PROBLEM == "classification":
    C = 1e0, 1e-1, 1e-3
    eta = 1e0
else:
    # C = 2e-5, 2e-3, 0
    C = 2e-3, 2e-4, 1e-4
    eta = 1e1
    
step_kwargs = {
    "C": C,                 # the regularizr constants (C_lasso, C_group, C_ridge)
    "eta": eta,             # the eta of the ADMM (larger - faster but more unstable)
    "rtol": 1e-5,           # the relative tolerance for stopping the ADMM
    "atol": 1e-8,           # the absolute tolerance
    "method": "cg",         # the method to use in Sub_0
    "n_iterations": 2,      # the number of iterations of the inner ADMM
}

In [5]:
# parameters for matrix sizes
n_samples, n_objects = 811, 915
n_rank = 26
n_features = 100
K = 25

# magnitude parameters
scale = 0.05
noise_to_signal = 0.15

# number of iterations
n_iter = 100

In [6]:
X, W_ideal, Y, H_ideal, R_noisy_full, R_clear_full = make_imc_data(
    n_samples, n_features, n_objects, n_features,
    n_rank, scale=(0.05, 0.05), noise=0.05*0.15,
    binarize=(PROBLEM == "classification"),
    random_state=random_state,
    return_noisy_only=False                                         # new parameter
)

# we use noisy data for training procedure
R_train, mask = sparsify(R_noisy_full, 0.10, random_state=random_state) 
problem = IMCProblem(QAObjectiveLoss, X, Y, R_train, n_threads=8)


W_0 = random_state.normal(size=(X.shape[1], K))
H_0 = random_state.normal(size=(Y.shape[1], K))

W, H = W_0.copy(), H_0.copy()

W, H = imc_descent(problem, W, H,
                   step_fn,                  # the inner optimization
                   step_kwargs=step_kwargs,  # asrtguments for the inner optimizer
                   n_iterations=n_iter,      # the number of outer iterations (Gauss-Siedel)
                   return_history=True,      # Record the evolution of the matrices (W, H)
                   rtol=1e-5,                # relative stopping tolerance for the outer iterations
                   atol=1e-7,                # absolute tolerance
                   verbose=True,             # show the progress bar
                   check_product=True,       # use the product W H' for stopping
                  )

100%|██████████| 100/100 [00:07<00:00, 15.95it/s]


In [7]:
mask_test = invert(mask)

print('Loss on noisy test:', calculate_loss(R_noisy_full, X, W, H, Y, mask_test))
print('Loss on clear test:', calculate_loss(R_clear_full, X, W, H, Y, mask_test))
print('Loss on noisy train:', calculate_loss(R_noisy_full, X, W, H, Y, mask))
print('Loss on clear train:', calculate_loss(R_clear_full, X, W, H, Y, mask))

Loss on noisy test: 0.288921020322
Loss on clear test: 0.0343073527558
Loss on noisy train: 0.271307777496
Loss on clear train: 0.032023181309
